In [1]:
import numpy as np
import pandas as pd
import random

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import os

In [2]:
#全局参数，随机种子，图像尺寸
seed = 114514
np.random.seed(seed)
random.seed(seed)
BATCH_SIZE = 512

hidden_dim = 16
epochs = 10
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cpu


## 数据准备

In [3]:
df = pd.read_csv('./train_dataset.csv')
print('共{}个用户，{}本图书，{}条记录'.format(max(df['user_id'])+1, max(df['item_id'])+1, len(df)))

共53424个用户，10000本图书，5869631条记录


In [4]:
df=df[:10000]

In [5]:
import tqdm
class Goodbooks(Dataset):
    def __init__(self, df, mode='training', negs = 99):
        super().__init__()

        self.df = df
        self.mode = mode

        self.book_nums = max(df['item_id'])+1
        self.user_nums = max(df['user_id'])+1

        self._init_dataset()
    
    def _init_dataset(self):
        self.Xs = []

        #下面的两个for循环建立了每一位用户与该用户看过的书籍之间的映射关系，即 :{用户1：用户1看过的书籍, 用户2：用户2看过的书籍, ...}
        self.user_book_map = {}
        for i in range(self.user_nums):
            self.user_book_map[i] = []
        for index, row in self.df.iterrows():
            user_id, book_id = row
            self.user_book_map[user_id].append(book_id)
            #self.user_book_map={用户1：用户1看过的书籍, 用户2：用户2看过的书籍, ...}
            
            
        #对于每一个用户的交互数据，训练集使用除了最后一个item之外的所有item(书籍)，而验证集只使用最后一个item(书籍)
        #训练集样本结构：(用户id，书籍id，label) 
        #label表示是否阅读，是：1，否：0
        if self.mode == 'training':
            for user, items in tqdm.tqdm(self.user_book_map.items()):
                for item in items[:-1]:
                    #构建正样本，对应label为1
                    self.Xs.append((user, item, 1))
                    #构建负样本，对应label为0
                    #正负样本比例为1:3，模拟真实情况下，用户已经阅读过的书籍数小于书籍总数
                    for _ in range(3):
                        while True:
                            neg_sample = random.randint(0, self.book_nums-1)
                            if neg_sample not in self.user_book_map[user]:
                                self.Xs.append((user, neg_sample, 0))
                                break
        #验证集样本结构：(用户id，已阅读书籍id，未阅读书籍id)
        elif self.mode == 'validation':
            for user, items in tqdm.tqdm(self.user_book_map.items()):
                if len(items) == 0:
                    continue
                self.Xs.append((user, items[-1]))
    
    def __getitem__(self, index):
        if self.mode == 'training':
            user_id, book_id, label = self.Xs[index]
            return user_id, book_id, label
        elif self.mode == 'validation':
            user_id, book_id = self.Xs[index]
            #在所有的当前用户没有看过的书籍中随机抽取99本，之后大概会对这99本排序，对当前用户进行推荐？
            negs = list(random.sample(
                list(set(range(self.book_nums)) - set(self.user_book_map[user_id])),
                k=99
            ))
            return user_id, book_id, torch.LongTensor(negs)
    
    def __len__(self):
        return len(self.Xs)

In [6]:
#建立训练和验证dataloader
traindataset = Goodbooks(df, 'training')
validdataset = Goodbooks(df, 'validation')

trainloader = DataLoader(traindataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=False, num_workers=0)
validloader = DataLoader(validdataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=False, num_workers=0)

100%|████████████████████████████████████████████████████████████████████████████████████████| 190/190 [00:00<?, ?it/s]


In [7]:
for i in traindataset:
    print(i)
    break

(0, 257, 1)


In [8]:
for i in trainloader:
    print(len(i))
    print(i[0].shape)#user id [512,1]
    print(i[1].shape)#book id(pos or neg都有) [512,1]
    print(i[2].shape)#label   [512,1]
    break

3
torch.Size([512])
torch.Size([512])
torch.Size([512])


In [9]:
for i in validdataset:
    print(i)
    break

(0, 900, tensor([6741, 4685, 6776, 3157, 3494, 9160, 9501, 4916, 1507, 9623, 1820, 4317,
        6840, 6515, 9953,  549, 3001, 2845, 3998, 8035, 4845, 4913, 4526, 4265,
        8189, 8291, 8728, 5992, 4564, 5559, 4418,  692, 6184, 7579, 9062, 1441,
        3790, 6852,  501, 8727, 8651, 3277, 9140,  831, 8127,  444, 8670, 5291,
         743, 1152, 8327, 9056, 7927, 6598, 4233, 8374,  488, 7187, 7063, 3064,
        3980, 2439, 9851, 3373, 4533, 7862, 3650, 2650, 5464, 7416, 7847, 3919,
        6250, 4627,  969,  466, 5746,  796, 1398, 8729, 3194, 8980, 6416, 7091,
         519,  537, 4869, 1064, 6744, 1384, 5467, 6204,  410, 6041, 3081, 5036,
        6912, 1499, 8367]))


In [10]:
for i in validloader:
    print(len(i))
    print(i[0].shape)#user id,[99,1]
    print(i[1].shape)#book id(pos),[99,1]
    print(i[2].shape)#book id(neg),[99,99]，负样本随机抽取了99条，具体见Goodbooks类
    break

3
torch.Size([99])
torch.Size([99])
torch.Size([99, 99])


## 模型构建

In [11]:
# 构建模型
class NCFModel(torch.nn.Module):
    def __init__(self, hidden_dim, user_num, item_num, mlp_layer_num=4, weight_decay = 1e-5, dropout=0.5):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.user_num = user_num
        self.item_num = item_num
        self.mlp_layer_num = mlp_layer_num
        self.weight_decay = weight_decay
        self.dropout=dropout
        
        #MLP的Embedding层
        self.mlp_user_embedding = torch.nn.Embedding(user_num, hidden_dim * (2 ** (self.mlp_layer_num - 1)))
        self.mlp_item_embedding = torch.nn.Embedding(item_num, hidden_dim * (2 ** (self.mlp_layer_num - 1)))
        
        #GMF的Embedding层
        self.gmf_user_embedding = torch.nn.Embedding(user_num, hidden_dim)    
        self.gmf_item_embedding = torch.nn.Embedding(item_num, hidden_dim)
        

        mlp_Layers = []
        input_size = int(hidden_dim*(2 ** (self.mlp_layer_num)))
        for i in range(self.mlp_layer_num):
            mlp_Layers.append(torch.nn.Linear(int(input_size), int(input_size / 2)))
            mlp_Layers.append(torch.nn.Dropout(self.dropout))
            mlp_Layers.append(torch.nn.ReLU())
            input_size /= 2
        self.mlp_layers = torch.nn.Sequential(*mlp_Layers)
        """
        Sequential(
          (0): Linear(in_features=256, out_features=128, bias=True)
          (1): Dropout(p=0.5, inplace=False)
          (2): ReLU()
          (3): Linear(in_features=128, out_features=64, bias=True)
          (4): Dropout(p=0.5, inplace=False)
          (5): ReLU()
          (6): Linear(in_features=64, out_features=32, bias=True)
          (7): Dropout(p=0.5, inplace=False)
          (8): ReLU()
          (9): Linear(in_features=32, out_features=16, bias=True)
          (10): Dropout(p=0.5, inplace=False)
          (11): ReLU()
        )
        """

        self.output_layer = torch.nn.Linear(2*self.hidden_dim, 1)

    def forward(self, user, item):
        
        user_gmf_embedding = self.gmf_user_embedding(user)
        item_gmf_embedding = self.gmf_item_embedding(item)
        
        user_mlp_embedding = self.mlp_user_embedding(user)
        item_mlp_embedding = self.mlp_item_embedding(item)

        #GMF执行element-wise product操作
        gmf_output = user_gmf_embedding * item_gmf_embedding
        
        #MLP块通过堆叠的全连接层+激活函数
        mlp_input = torch.cat([user_mlp_embedding, item_mlp_embedding], dim=-1)
        mlp_output = self.mlp_layers(mlp_input)
        
        #将GMF和MLP的输出结果concat起来，送入最后的全连接层预测结果，并使用sigmoid函数将输出结果映射到0与1之间
        output = torch.sigmoid(self.output_layer(torch.cat([gmf_output, mlp_output], dim=-1))).squeeze(-1)
        
        return output
    
    def predict(self, user, item):
        self.eval()
        #print(user.shape,item.shape)#torch.Size([512]) torch.Size([512, 100])
        with torch.no_grad():
            user_gmf_embedding = self.gmf_user_embedding(user)
            item_gmf_embedding = self.gmf_item_embedding(item)

            user_mlp_embedding = self.mlp_user_embedding(user)
            item_mlp_embedding = self.mlp_item_embedding(item)

            gmf_output = user_gmf_embedding.unsqueeze(1) * item_gmf_embedding

            user_mlp_embedding = user_mlp_embedding.unsqueeze(1).expand(-1, item_mlp_embedding.shape[1], -1)#[512, 128]->[512,1,128]->[512,100,128]
            mlp_input = torch.cat([user_mlp_embedding, item_mlp_embedding], dim=-1)
            mlp_output = self.mlp_layers(mlp_input)

        output = torch.sigmoid(self.output_layer(torch.cat([gmf_output, mlp_output], dim=-1))).squeeze(-1)
        return output

## 模型训练&模型评估

In [12]:
model = NCFModel(hidden_dim, traindataset.user_nums, traindataset.book_nums).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
crit = torch.nn.BCELoss()

loss_for_plot = []
hits_for_plot = []

for epoch in range(epochs):
    #训练
    losses = []
    for index, data in enumerate(tqdm.tqdm(trainloader)):
        user, item, label = data
        user, item, label = user.to(device), item.to(device), label.to(device).float()
        y_ = model(user, item).squeeze()

        loss = crit(y_, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.detach().cpu().item())
        
    #验证
    hits = []
    for index, data in enumerate(validloader):
        user, pos, neg = data
        #print(pos.shape,neg.shape)#torch.Size([512]) torch.Size([512, 99])
        pos = pos.unsqueeze(1)#[512->[512,1]
        all_data = torch.cat([pos, neg], dim=-1)
        #print(all_data)#torch.Size([512, 100])
        output = model.predict(user.to(device), all_data.to(device)).detach().cpu()##torch.Size([512, 100])
        
        for batch in output:
            #print('batch',batch)
            pred10=(batch).argsort(descending=True)[:10]#从大到小，取前10所在下标
            
            print('pred10',pred10)
            
            #索引0是正样本，如果预测的前10中没有0，那么说明预测错了
            if 0 not in pred10:
                hits.append(0)
            else:
                hits.append(1)
                
    print('Epoch {} finished, average loss {}, hits@20 {}'.format(epoch, sum(losses)/len(losses), sum(hits)/len(hits)))
    loss_for_plot.append(sum(losses)/len(losses))
    hits_for_plot.append(sum(hits)/len(hits))

  0%|                                                                                           | 0/78 [00:00<?, ?it/s]

tensor([[6813, 6870, 8355,  ..., 3921, 4500, 9685],
        [ 170, 6294, 7788,  ..., 9411, 1451, 4902],
        [2840, 8702, 4207,  ..., 1121, 3270, 1666],
        ...,
        [ 154, 1136, 8977,  ..., 5602, 7667, 2621],
        [1872, 9289, 1737,  ..., 2925, 8020, 2848],
        [9511, 4276, 3353,  ..., 2549, 7756, 2935]])
pred10 tensor([74, 26,  2, 34,  5, 62, 28, 75, 98, 96])
pred10 tensor([56, 37, 92, 42, 15, 31, 93, 83, 14, 51])
pred10 tensor([39, 83, 96, 11, 56, 87, 22, 45, 72, 97])
pred10 tensor([34, 67, 71, 38, 26, 28, 39, 45, 41, 74])
pred10 tensor([13, 20, 48, 36, 80, 53, 65, 97, 41, 22])
pred10 tensor([47,  2, 41, 31, 74, 26, 49, 18, 51, 65])
pred10 tensor([62, 57, 72, 83, 34, 85, 27,  7, 74, 76])
pred10 tensor([79, 85, 19, 63, 73, 41,  3, 18, 74, 86])
pred10 tensor([ 7, 31, 75, 63,  6, 32,  1, 52,  5, 89])
pred10 tensor([95, 28, 22, 50, 45, 76, 66, 37,  5, 77])
pred10 tensor([66, 71, 78, 64, 21, 68, 74, 46, 67, 23])
pred10 tensor([86, 35, 90, 71, 34, 99, 26, 23, 38, 36])
pr

  0%|                                                                                           | 0/78 [00:00<?, ?it/s]

tensor([[1075, 1027, 5295,  ..., 1367, 7672, 9779],
        [2746, 5305, 5662,  ..., 5979, 5063, 5695],
        [ 704,  942,   33,  ..., 9091, 5837, 9216],
        ...,
        [5493, 1227, 7454,  ..., 4598, 1565, 2108],
        [6798, 3145, 9804,  ..., 2063, 7461,  397],
        [1450, 8128, 1182,  ..., 9559, 7111, 7104]])
pred10 tensor([90, 45, 44, 55, 19, 37, 38, 65, 39, 83])
pred10 tensor([33,  9, 30, 60, 59, 70, 23, 35, 52, 56])
pred10 tensor([89,  8, 81, 86, 80,  1,  4, 42, 24, 45])
pred10 tensor([51, 16, 53, 77, 13, 85, 30, 76, 81, 50])
pred10 tensor([79, 71,  0, 46, 95, 50, 53, 92,  7, 32])
pred10 tensor([28, 49, 35, 92, 58, 11, 51, 23, 99,  5])
pred10 tensor([38, 21, 32, 12, 47,  4, 59, 79, 81, 65])
pred10 tensor([37, 67, 56, 64, 52, 92, 70, 63, 12, 84])
pred10 tensor([42, 30, 84, 43, 44, 39, 86, 36, 85, 18])
pred10 tensor([99, 19, 82, 88,  0,  8, 49, 22, 28, 76])
pred10 tensor([85,  9, 60, 89, 34, 68, 82,  1, 47,  2])
pred10 tensor([ 9, 82, 74, 79, 50, 29, 14, 86, 22, 32])
pr

  0%|                                                                                           | 0/78 [00:00<?, ?it/s]

...

tensor([[3645, 5423, 6339,  ..., 5982, 4254, 6940],
        [5458, 3009, 3235,  ..., 4715, 4778, 4879],
        [ 982, 8220, 8735,  ..., 5032, 6200, 3032],
        ...,
        [ 396, 7073, 9736,  ..., 2851, 6254, 5803],
        [9511,  274, 4647,  ..., 8407, 8080, 3293],
        [ 249, 8164,  546,  ..., 6755,  325, 4478]])
pred10 tensor([79,  9, 65, 58, 34, 80, 42, 72,  3,  7])
pred10 tensor([16,  3, 56, 26, 75, 34, 12, 76, 13, 24])
pred10 tensor([72, 91, 75, 78, 92, 45, 21, 56, 47, 93])
pred10 tensor([92, 36, 69,  6, 96, 37, 74, 56, 19, 34])
pred10 tensor([ 0, 54, 52, 84, 53,  1, 92, 78, 62, 28])
pred10 tensor([57, 23, 59,  1, 45, 28, 64, 12, 36, 76])
pred10 tensor([56,  8, 97, 19, 87, 60, 62,  6, 66, 43])
pred10 tensor([71, 51,  0, 88, 73, 64, 43, 97, 16, 63])
pred10 tensor([59, 74, 82, 53, 87, 33,  6, 94, 47, 38])
pred10 tensor([92, 37,  7, 55, 57,  0, 63,  2, 84, 24])
pred10 tensor([56, 95, 57, 26, 54, 19, 96, 71, 82,  0])
pred10 tensor([94, 36, 28, 74, 29, 43, 76, 59,  9, 53])
pr

  0%|                                                                                           | 0/78 [00:00<?, ?it/s]

tensor([[ 113, 1961, 5517,  ..., 9282, 2943, 1664],
        [ 919, 3461, 2369,  ..., 9981, 3632, 5923],
        [  20, 7422, 6854,  ..., 5508, 2586, 6690],
        ...,
        [ 397, 6077, 5916,  ...,  652, 1320, 6387],
        [ 982, 8262, 7777,  ..., 8282, 8114, 6578],
        [  13, 2624, 6107,  ..., 3843, 4709, 1814]])
pred10 tensor([34, 55,  6,  0, 95, 37, 66, 25, 76, 98])
pred10 tensor([69, 20, 15, 34, 32, 81, 31, 93, 44, 80])
pred10 tensor([ 0, 49, 61, 21,  4, 46, 63, 29, 76,  7])
pred10 tensor([19, 63, 52, 64, 61, 58, 97,  0, 15, 59])
pred10 tensor([67, 36,  4, 38, 52, 27, 98, 34, 70,  8])
pred10 tensor([74,  0, 75,  6, 65, 42, 70,  4, 38, 53])
pred10 tensor([ 6, 33,  0, 45, 16, 23, 77, 70, 78, 80])
pred10 tensor([45, 65, 79, 58, 27, 14, 21, 20,  0, 28])
pred10 tensor([75, 85, 65, 43, 70, 79, 41,  2, 89, 56])
pred10 tensor([10, 86, 18, 28, 88,  9, 62,  0, 74, 70])
pred10 tensor([63, 72, 97,  4, 16, 96,  3, 39, 43, 50])
pred10 tensor([75, 37, 79, 59, 12, 34, 45, 68, 93, 56])
pr

100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:02<00:00, 28.83it/s]


tensor([[ 365, 3506, 2860,  ..., 6413, 9968, 3771],
        [ 851,  758, 2326,  ...,  317,  210, 6984],
        [ 384,  354, 2299,  ..., 5416, 9370, 1986],
        ...,
        [2176, 9866, 3904,  ..., 6307, 3283, 5332],
        [ 284, 8515, 6282,  ..., 8316, 3443, 6406],
        [1075, 7281, 9909,  ...,  148, 2451, 8487]])
pred10 tensor([73, 58, 18, 11, 91, 83, 21, 74,  0, 90])
pred10 tensor([98,  5, 33, 80, 34, 72, 45, 27, 12, 10])
pred10 tensor([42, 76, 19, 89, 16, 21, 67, 41, 13, 69])
pred10 tensor([13, 81, 31,  0,  7,  4, 40, 15, 96, 75])
pred10 tensor([11, 10, 13, 47, 84, 78, 64, 15, 27, 39])
pred10 tensor([22, 97, 31, 66, 87, 68, 55, 90, 45, 33])
pred10 tensor([85, 10,  1, 17, 58, 66, 88,  3, 86, 51])
pred10 tensor([ 4, 91, 49, 23, 57, 53, 42, 77, 51, 27])
pred10 tensor([27, 16, 42, 71, 38, 30, 11, 31, 33, 14])
pred10 tensor([24, 34, 42, 31, 87,  7, 97, 73, 96, 20])
pred10 tensor([18, 14, 93, 77, 76, 91, 61, 23,  8,  0])
pred10 tensor([50, 11, 14, 48, 52,  1,  7, 62, 80,  8])
pr

In [14]:
model = NCFModel(hidden_dim, traindataset.user_nums, traindataset.book_nums).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
crit = torch.nn.BCELoss()

loss_for_plot = []
hits_for_plot = []

for epoch in range(epochs):
    #训练
    losses = []
    for index, data in enumerate(tqdm.tqdm(trainloader)):
        user, item, label = data
        user, item, label = user.to(device), item.to(device), label.to(device).float()
        y_ = model(user, item).squeeze()

        loss = crit(y_, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.detach().cpu().item())
        
    #验证
    hits = []
    for index, data in enumerate(validloader):
        user, pos, neg = data
        #print(pos.shape,neg.shape)#torch.Size([512]) torch.Size([512, 99])
        pos = pos.unsqueeze(1)#[512->[512,1]
        all_data = torch.cat([pos, neg], dim=-1)
        print(all_data)#torch.Size([512, 100])
        output = model.predict(user.to(device), all_data.to(device)).detach().cpu()##torch.Size([512, 100])
        
        #每一个用户预测的结果对应output中的一行(batch),被预测的相应item是all_data中的一行(batch_items)
        for batch ,batch_items in zip(output,all_data):
            
            pos_id=batch_items[0]#取出正样本对应的真实的item id
            
            pred10=(batch).argsort(descending=True)[:10]#预测值从大到小，取前10所在下标
            pred10=batch_items[pred10]#在batch_items中的真实下标，这才是item id
            print(pred10)
            #索引0是正样本，如果预测的前10中没有0，那么说明预测错了
            if pos_id not in pred10:
                hits.append(0)
            else:
                hits.append(1)
                
    print('Epoch {} finished, average loss {}, hits@20 {}'.format(epoch, sum(losses)/len(losses), sum(hits)/len(hits)))
    loss_for_plot.append(sum(losses)/len(losses))
    hits_for_plot.append(sum(hits)/len(hits))

  0%|                                                                                           | 0/78 [00:00<?, ?it/s]

tensor([[ 154, 7793, 7855,  ..., 5052,  441, 7805],
        [1728, 6290, 2170,  ..., 9121, 2102, 1346],
        [4530, 9667, 4790,  ..., 2918, 3704, 5545],
        ...,
        [7484, 3735, 8418,  ..., 5728, 9618,  907],
        [3435, 1830, 2604,  ..., 2331, 8905, 7993],
        [1075,  334, 8454,  ..., 3879, 6621, 8711]])
tensor([5677, 4348, 2675, 6947, 4948, 8039, 2001,  208, 6318, 9099])
tensor([ 353, 1878,  649, 8140, 2729, 6613, 4014, 1544, 2934,  523])
tensor([4790, 6921, 1173, 4973, 9620, 8479, 5804, 3704,  713, 8636])
tensor([4173, 8114, 3897, 4782, 6409, 3607, 3382, 5489, 6833, 8312])
tensor([5677, 8113, 7870, 7722, 5212, 3121, 1765, 4472, 8023,  153])
tensor([4833, 8267, 6679, 5005,  781, 4788, 5440, 2842, 3422, 9678])
tensor([5344, 1533, 8994, 2059, 3106, 2460, 8535, 1337, 2167,  960])
tensor([3203, 3411, 6204, 1210, 9172, 4823, 3966, 5366, 7802, 4574])
tensor([1491, 4739, 9315, 1723, 5056, 6627, 9451, 1950, 1170, 6453])
tensor([4617, 9320, 5266, 7584, 8036, 9847, 6385, 778

  0%|                                                                                           | 0/78 [00:00<?, ?it/s]

tensor([[ 967, 7831, 7030,  ..., 4745, 3048, 9366],
        [2246, 3153, 4626,  ..., 9514, 7469, 1913],
        [ 166, 3003, 1714,  ..., 7968, 5834, 2264],
        ...,
        [ 115, 1850, 5299,  ..., 1237,  287, 1493],
        [3435, 7733,  883,  ..., 7343, 5614, 6687],
        [ 701,  990, 1047,  ..., 2278, 2231,  131]])
tensor([5449, 4274,  597, 3061, 1305, 3579, 7536, 6379, 4185, 9315])
tensor([7149,  940, 8841, 8558, 4198, 3070, 5207, 9716, 6979, 8724])
tensor([5439, 4835, 1714, 5921, 6835, 2662,  575, 7952, 2717, 4228])
tensor([3794, 4520, 2774, 4465, 5882, 7527, 4190, 2060, 2517, 3708])
tensor([9223, 1979, 3119,  941,  800, 8843,   49, 2116, 7149, 9570])
tensor([5213, 4393, 6190,  652, 2450, 7955, 7926, 7637,  329, 2423])
tensor([5509, 4150,   20, 1574, 3874, 6871, 8374, 8584, 1981, 5774])
tensor([2928, 6990, 6089, 1987, 6841,  696, 4072, 9077, 4310, 4128])
tensor([  61, 8603, 5555, 8457,  714, 1591, 7253, 5054,   49, 4525])
tensor([5082,  652,   26,  742,   65,  556, 2354,  27

  0%|                                                                                           | 0/78 [00:00<?, ?it/s]

...

tensor([[2840, 4998, 6427,  ...,   23, 3799, 9463],
        [ 141,  158,  456,  ..., 4388, 3832, 4098],
        [ 984, 9956, 6073,  ..., 7797, 4358, 7310],
        ...,
        [ 123, 7033, 5860,  ..., 3568, 3786, 3192],
        [2246,  990, 6133,  ..., 9197, 8268, 8062],
        [ 145,  167, 3754,  ..., 8913, 1938, 7512]])
tensor([  23,   83, 4116,  837,  446,  594, 6943, 1459, 6560,  491])
tensor([ 546,  118, 5729,  141, 5829,  772,  158, 1260, 1083, 1441])
tensor([ 940,  157, 2452, 8430,  954,  665,  220, 2631, 4066, 5325])
tensor([ 915,  958, 2026, 4019, 9203, 9525, 1857, 1543,  394, 5733])
tensor([  93, 1205,  762, 9486, 2048, 1662,  229, 5207,  849, 5742])
tensor([1328,  491,  269, 7007, 2925, 6643, 5664, 2890, 1245, 3645])
tensor([ 940, 2534,   12, 3645,  174,  208, 3356,   85,  704, 7359])
tensor([1298,  653, 7872, 2461, 3585,  704, 8123, 5384, 2081, 1760])
tensor([ 123,   47, 2297, 4605, 4941, 9245, 4745, 7252,  658, 5118])
tensor([ 179,   19,  128, 7002, 2097, 3510, 2238, 185

  0%|                                                                                           | 0/78 [00:00<?, ?it/s]

tensor([[6813, 4262,  458,  ..., 8237, 6255, 6765],
        [1724, 6330,  719,  ...,  308,  876, 4783],
        [ 145, 3341, 9280,  ..., 3484, 6730, 6198],
        ...,
        [ 898, 4918, 3809,  ..., 7461, 8352, 8316],
        [ 123, 2508, 4598,  ..., 8667,  899,  584],
        [2610, 9385, 7522,  ..., 3512, 2825, 5061]])
tensor([ 255, 2102,  207,  713, 1013, 4099, 1282, 3432, 6628,  150])
tensor([1017, 4019, 6819,  345, 5445, 7765, 8603, 6684,  701,  366])
tensor([ 537,   94,  145,  303, 1010, 1219, 4613, 8877, 2492, 5708])
tensor([ 489,  214, 1579,  781, 6104, 5345,  957,  131,   14, 3450])
tensor([ 521, 1264,  140,   21,   31, 1955, 7952,  221, 5422, 2008])
tensor([ 605, 9077, 2023, 7285, 6480,  680, 9857, 1047, 6560, 1576])
tensor([8881, 3378, 2251, 7150, 1023, 7918, 7092,  742,  968,  727])
tensor([ 555,   35,  942,  489,   13, 1968,  750, 1348, 2545, 2088])
tensor([ 981,  147,  714, 2317, 1251,  188, 2673, 5599, 1508,  814])
tensor([  22,  159,  172,  180, 1537,  871, 4725,  68

100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:02<00:00, 28.83it/s]


tensor([[1075, 8410, 7752,  ..., 5009, 3269, 6746],
        [ 378,  771, 3739,  ..., 4092, 9982, 1286],
        [ 384, 3259, 5066,  ..., 7196, 5458, 1329],
        ...,
        [1728, 1401, 6244,  ..., 2167, 7356, 2169],
        [2484, 8529, 6265,  ..., 4611,  630, 5468],
        [ 704, 6725, 7076,  ..., 5371, 6614, 4493]])
tensor([2317, 5490,  187, 1576,    6, 8653, 8020, 2549, 1025, 2673])
tensor([ 371, 6363,   95, 1040,  153, 7521, 8948, 2672,  340, 9587])
tensor([ 397, 7196, 2614,   42,  260,  948, 4744, 7918, 7480, 4827])
tensor([5524,  309, 2448, 1130,  242,   43, 8408, 3605,  974, 7839])
tensor([2346, 8835,  261,  224, 9301,  948, 3509, 6011, 6585, 2715])
tensor([  80, 8454,  969, 7873,  918, 9772,   33, 4254,  277,  436])
tensor([7261,  407,  102, 2662, 6593,  107, 2902, 4028, 1194,  922])
tensor([1153,  341,  952, 8108, 1514,  968, 8383, 5571, 6633, 9742])
tensor([ 765, 1074,  116,   91, 4393,  139, 2189,  360, 1499, 8776])
tensor([  17, 1004, 2797, 2780,   99, 1517, 1135,   9

In [ ]:
# 模型保存
torch.save(model.state_dict(), './model.h5')

## 模型预测

In [16]:
df = pd.read_csv('./test_dataset.csv')
user_for_test = df['user_id'].tolist()

In [19]:
user_for_test

[0,
 1,
 3,
 5,
 7,
 8,
 10,
 14,
 17,
 21,
 23,
 24,
 27,
 31,
 28,
 33,
 39,
 30,
 42,
 48,
 45,
 49,
 52,
 54,
 57,
 60,
 61,
 64,
 65,
 53,
 55,
 68,
 69,
 62,
 71,
 72,
 58,
 73,
 74,
 75,
 79,
 37,
 82,
 84,
 77,
 44,
 88,
 92,
 93,
 99,
 25,
 102,
 104,
 107,
 111,
 112,
 113,
 114,
 116,
 85,
 89,
 115,
 122,
 123,
 124,
 125,
 127,
 128,
 134,
 121,
 136,
 135,
 140,
 141,
 142,
 148,
 150,
 155,
 129,
 157,
 163,
 35,
 166,
 168,
 172,
 173,
 174,
 176,
 175,
 179,
 177,
 182,
 170,
 183,
 184,
 186,
 188,
 167,
 189,
 105]

In [26]:
predict_item_id = []

def chunks(l, n):#n: batch size
    for i in range(0, len(l), n):
        yield l[i:i+n]

f = open('./submission.csv', 'w', encoding='utf-8')

#预测每一个用户user可能会点击的图书item
for user in tqdm.tqdm(user_for_test):
    #将用户已经交互过的物品排除
    user_visited_items = traindataset.user_book_map[user]
    items_for_predict = list(set(range(traindataset.book_nums)) - set(user_visited_items))

    results = []
    user = torch.Tensor([user]).to(device).long()

    for item_batch in chunks(items_for_predict, 512):
        item_batch = torch.Tensor(item_batch).unsqueeze(0).to(device).long()
        
        #print(user.shape)#torch.Size([1])
        #print(item_batch.shape)#torch.Size([1, 512])

        result = model.predict(user, item_batch).view(-1).detach().cpu()
        #print(result.shape)#torch.Size([512])
        results.append(result)
    #print(len(results),len(results[0]))#(20,512),注意results[-1]不一定是512，因为可能不足一个batch (batch size =512)
    results = torch.cat(results, dim=-1)#所有items_for_predict关于用户user的预测值

    #取得分前10的item在results(也在items_for_predict)中的下标
    predict_item_id = results.argsort(descending=True)[:10]#从大到小排序，取前10
    print('ind:',predict_item_id)
    
    #映射到真实的item id
    res=[]
    for i in predict_item_id:
        res.append(items_for_predict[i])
    print('res:',res)
    list(map(lambda x: f.write('{},{}\n'.format(user.cpu().item(), x)), predict_item_id))

f.flush()
f.close()

  3%|██▍                                                                               | 3/100 [00:00<00:04, 19.42it/s]

ind: tensor([  0,  75, 507,  82,  18, 301,  76,  12,  19,   6])
res: [0, 111, 586, 120, 24, 371, 112, 17, 25, 7]
ind: tensor([   0,  550,  336, 1111,   42,   93, 4308,  534,    6,   76])
res: [0, 586, 371, 1153, 57, 111, 4360, 569, 10, 93]
ind: tensor([  0, 486, 172, 586, 235, 289,  63,  71,  58, 231])
res: [0, 586, 237, 690, 309, 371, 111, 123, 105, 305]
ind: 

  7%|█████▋                                                                            | 7/100 [00:00<00:05, 17.03it/s]

tensor([   0,  114,    7,  154, 1211,   18,  667,   56,   17,   77])
res: [0, 121, 7, 161, 1255, 18, 700, 57, 17, 80]
ind: tensor([3345, 1323,  139, 5990,    0,  544,   47,  212,   15,  186])
res: [3424, 1384, 159, 6077, 0, 586, 57, 237, 20, 210]
ind: tensor([   3,   62,   27,   71,   28,   12,    0,  113, 2884,  191])
res: [7, 99, 56, 111, 57, 34, 3, 158, 3001, 248]
ind: tensor([   1,    0, 1305,  128,   41,  945,   12,  126,   86,  448])
res: [1, 0, 1384, 161, 57, 1020, 17, 159, 112, 508]


  9%|███████▍                                                                          | 9/100 [00:00<00:05, 15.97it/s]

...

ind: tensor([   0,   35, 1501,   43,  153, 1321,   14,  497,   12,   15])
res: [0, 48, 1565, 57, 180, 1384, 19, 545, 16, 20]
ind: tensor([  43,  276,  370,   86,   15,  136,  647, 1199,  902,  624])
res: [53, 305, 404, 101, 23, 159, 690, 1255, 952, 667]
ind: tensor([   0,  521, 1180, 9510,  117, 1715,    4, 6965, 1078,  218])
res: [0, 586, 1255, 9605, 160, 1795, 9, 7058, 1153, 268]


 98%|███████████████████████████████████████████████████████████████████████████████▍ | 98/100 [00:06<00:00, 13.64it/s]

ind: tensor([   0,    8,   43, 1406,    6, 2450,  541, 9351,   17, 1314])
res: [0, 10, 53, 1478, 7, 2526, 586, 9437, 20, 1384]
ind: tensor([ 648,    6,    0, 1486, 2121,  297,  907, 6513,  281, 5774])
res: [690, 7, 0, 1536, 2176, 322, 952, 6592, 305, 5850]
ind: tensor([   0,   57,    7,  126,   17, 1701,  263,   48,  179,  100])
res: [0, 57, 7, 126, 17, 1702, 263, 48, 179, 100]


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:06<00:00, 14.94it/s]

ind: tensor([  0,  25,   7, 371, 120,  57,  10,  23,  17, 161])
res: [0, 25, 7, 371, 120, 57, 10, 23, 17, 161]


In [24]:
predict_item_id

tensor([  0,  25,   7, 371, 120,  57,  10,  23,  17, 161])

In [25]:
res

[0, 25, 7, 371, 120, 57, 10, 23, 17, 161]